In [1]:
def RSI(series, period = 5):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

In [2]:
def SOI(ds,priod=14):
    ds['14-high'] = ds['high'].rolling(priod).max()
    ds['14-low'] = ds['low'].rolling(priod).min()
    ds['K'] = (ds['14-low'])*100/(ds['14-high'] - ds['14-low'])
    ds['D'] = ds['K'].rolling(3).mean()
    return ds

In [3]:
def SMA(series,w = 3):
    return series.rolling(window=w).mean()

In [4]:
def CMA(series,mp=4):
    return series.expanding(min_periods=mp).mean()

In [5]:
def EMA(series,sp=40):
    return series.ewm(span=sp,adjust=False).mean()

In [6]:
import matplotlib
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.naive_bayes import *
from sklearn.linear_model import LinearRegression
from re import sub
def mdy_to_ymd(d):
    return time.mktime(datetime.strptime(d, '%d-%b-%y').timetuple())

class Beautify:

    @staticmethod
    def beautify(path):
        dataset = pd.read_csv(path)
        dataset.columns = ["date", "price", "open", "high", "low", "vol", "change"]
        dataset["date"] = dataset.apply(lambda row: mdy_to_ymd(row['date']), axis=1)
        for col in ["price", "open", "high", "low", "vol", "change"]:
            dataset[col] = dataset.apply(
                lambda row: (lambda x: 0 if x == '-' else float(x))(sub(r'[^\d.-]', '', row[col])),
                axis=1)
        dataset = dataset.sort_values(["date"])
        return dataset


pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [7]:
dataset = SOI(Beautify.beautify("dataset.csv"))
dataset["RSI"] = RSI(dataset.price)
dataset["SMA"] = SMA(dataset.price)
dataset["CMA"] = CMA(dataset.price)
dataset["EMA"] = EMA(dataset.price)

In [8]:
for name in ["14-high","14-low","K","D","RSI","SMA","CMA","EMA"]:
    dataset[name] = dataset[name].fillna(0)

In [9]:
dataset = dataset.drop(["K","open","high","low","vol","change"],axis=1)

In [10]:
for i in [1,2,3,5,7,10,12,15,20,30,45]:
    dataset[f"s_{i}"] = dataset["price"].shift(i,fill_value=0)

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

In [12]:
TRAIN = time.mktime(datetime.strptime("20-01-02", '%y-%m-%d').timetuple())
train = dataset[dataset.date < TRAIN]
test = dataset[dataset.date >= TRAIN]

In [13]:
train.drop("date",axis=1,inplace=True)
test.drop("date",axis=1,inplace=True)

E:\Anaconda2021\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [14]:
model = LinearRegression()
model.fit(train.drop("price",axis=1),train.price)
pred = model.predict(test.drop("price",axis=1))

In [15]:
res = pd.DataFrame({"index": np.arange(len(pred)), "real": test.price, "pred": pred})

In [16]:
res[(res.pred * 0.95 <= res.real) & (res.pred * 1.05 >= res.real)].shape[0] / res.shape[0] * 100

100.0

In [17]:
from sklearn.metrics import mean_squared_error
mean_squared_error(res.pred,res.real)

1.3383895372576343e-07